#### Imports

In [1]:
import logging
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
logging.basicConfig(level=logging.DEBUG)

In [3]:
def _get_response(url):
    response = requests.get(url)
    return response

In [4]:
def get_soup_object(url):
    response = _get_response(url)
    #Insert logic to check
    #if response = 200
    soup = BeautifulSoup(response.text, "html.parser")
    #else handle exceptions
    return soup

In [5]:
def get_aircraft_urls(url, aircraft_url_input_dir):
    df_aircraft_urls = pd.read_table(aircraft_url_input_dir)
    aircraft_url_list = []
    for index in range(len(df_aircraft_urls)):
        url_suffix_aircraft = df_aircraft_urls['URL'][index]
        aircraft_url_list.append(f'{url}{url_suffix_aircraft}')
        logging.debug(f'The URL being read in from csv is: {url}{url_suffix_aircraft}')
    return df_aircraft_urls, aircraft_url_list

In [6]:
def _read_table(table_number, soup):
    table = soup.find_all('table')[table_number-1]
    rows = table.find_all('tr')
    row_list = list()
    for tr in rows:
        td = tr.find_all('td')
        row = [i.text for i in td]
        row_list.append(row)
    flat_list = [item for sublist in row_list for item in sublist]
    return flat_list

In [7]:
def read_table_one(soup):
    
    table_number = 1
    
    flat_list = _read_table(table_number, soup)
    
    _list = []
    dict_keys = []
    dict_values = []
    
    flat_list = flat_list[:13] #Specific - change them ???????????
    
    for i in range(len(flat_list)):
        _list_val = flat_list[i]
        _sublist = _list_val.split(': ') #Specific to this context
        _list.append(_sublist)

    for i in range(len(_list)):
        dict_keys.append(_list[i][0])
        dict_values.append(_list[i][1])
    
    _dict = dict(zip(dict_keys, dict_values))
    
    df_table_1 = pd.DataFrame().append(_dict, ignore_index=True)
    
    return df_table_1

In [8]:
def read_table_two(soup):
    
    table_number = 2
    
    flat_list = _read_table(table_number, soup)
    
    _list = []
    dict_keys = []
    dict_values = []
    
    for i in range(len(flat_list)):
        _list_val = flat_list[i]
        _sublist_2 = _list_val.split(' Max Range: ')
        _sublist = _sublist_2[0].split('\n\t\t\t\t\t\t')
        _sublist.append(_sublist_2[1]) #Specific to this context
        _list.append(_sublist)
    
    for i in range(len(_list)):
        dict_keys_template = [f'Sensor {i+1} Name', f'Sensor {i+1} Type', f'Sensor {i+1} Max Range']
        for j in range(len(_list[i])):
            dict_keys.append(dict_keys_template[j])
            dict_values.append(_list[i][j])
    
    _dict = dict(zip(dict_keys, dict_values))
    
    df_table_2 = pd.DataFrame().append(_dict, ignore_index=True)
    
    return df_table_2

In [9]:
def read_table_three(soup):
    
    table_number = 3
    
    flat_list = _read_table(table_number, soup)
    
    _list = []
    dict_keys = []
    dict_values = []
    
    #insert logic here for table 3
    
    _dict = dict(zip(dict_keys, dict_values))
    
    df_table_3 = pd.DataFrame().append(_dict, ignore_index=True)
    
    return df_table_3

In [10]:
# def read_aircraft_data():
    

In [11]:
def read_image(soup, url):
    images = soup.findAll('img')
    #insert logic to choose correct index of image from images list
    image_url = images[1]['src']
    complete_url = f'{url}{image_url}'
    
    #perhaps insert try catch logic here to cater for exceptions
    image_data = requests.get(complete_url).content
    
    logging.debug(f'Image data recieved from URL.')
    
    return image_data

In [12]:
def save_image(image_data, image_output_dir, image_name, image_type):
    with open(f'{image_output_dir}{image_name}.{image_type}', 'wb') as handler:
        handler.write(img_data)
    logging.debug(f'Image written to file: {image_output_dir}{image_name}.{image_type}')

In [13]:
# def combine_tables_into_dataframe(df_table_1, df_table_2, df_table_3):
def combine_tables_into_dataframe(df_table_1, df_table_2):
#     Something along these lines - but better naming convention:
#     aircraft_data['Name'] = df_aircraft['Link'][0]
#     aircraft_data = pd.concat([df_table_1, df_table_2, df_table_3], axis=1, sort=False)
    aircraft_data = pd.concat([df_table_1, df_table_2], axis=1, sort=False)
    
    return aircraft_data

In [14]:
def write_dataframe_to_file(dataframe, csv_output_dir, csv_output_filename):
    logging.debug(f'...File {csv_output_filename}.csv is being written to disk...')
    dataframe.to_csv(f'{csv_output_dir}{csv_output_filename}.csv', ignore_index=True)
    logging.debug(f'Writing complete.')

In [15]:
def main():
    aircraft_url_input_dir = '../data/01_raw/aircraft_url.txt'
    csv_output_dir = '../data/02_intermediate/'
    csv_output_filename = 'aircraft_data'
    image_output_dir = '../data/01_raw/weapons_images/'
    image_type = 'jpg'
    root_url = 'http://cmano-db.com/'
    
    df_aircraft_urls, aircraft_url_list = get_aircraft_urls(root_url, aircraft_url_input_dir)
    
    aircraft_data = pd.DataFrame()
    
    for index in range(len(aircraft_url_list)):
        
        url = aircraft_url_list[index]
        
        logging.debug(f'Current URL being processed is: {url} at index: {index}.')
        soup = get_soup_object(url)

        df_table_1 = read_table_one(soup)
        logging.debug(f'Table 1 returned successfully.')
        df_table_2 = read_table_two(soup)
        logging.debug(f'Table 2 returned successfully.')
#         df_table_3 = read_table_three(soup)
    
#         _aircraft_data = combine_tables_into_dataframe(df_table_1, df_table_2, df_table_3)
        _aircraft_data = combine_tables_into_dataframe(df_table_1, df_table_2)
        logging.debug(f'Table 1 and 2 combined successfully.')
        aircraft_data = pd.concat([aircraft_data, _aircraft_data])
        logging.debug(f'Row {index} appended to dataframe.')
    
    write_dataframe_to_file(aircraft_data, csv_output_dir, csv_output_filename)
    
#     for 
#     image_name = df_aircraft_urls[]
#     image_data = read_image(soup, url)
#     save_image(image_data, image_output_dir, image_name, image_type)

In [16]:
if __name__ == '__main__':
    main()

DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/4354/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3040/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3137/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/4785/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/4786/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1341/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1656/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/4538/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/4537/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1657/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1386/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/4583/
DEBUG:root:The URL being rea

DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3245/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3247/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3246/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1701/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1782/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3204/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/276/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3179/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3316/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3449/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1521/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/940/
DEBUG:root:The URL being read 

DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/731/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/734/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1997/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1998/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1999/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/2248/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3249/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3258/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3259/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3260/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3261/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3262/
DEBUG:root:The URL being read 

DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3108/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3859/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/4074/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/4073/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1794/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/574/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1269/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/578/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/4610/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/2779/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/648/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/2983/
DEBUG:root:The URL being read i

DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3308/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1730/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/2298/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/823/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1729/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/2172/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/2503/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3727/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/715/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/4633/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/2478/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3300/
DEBUG:root:The URL being read 

...

DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/666/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/693/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/704/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/860/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3813/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/46/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/325/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/552/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/553/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/861/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1921/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/416/
DEBUG:root:The URL being read in from c

DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/425/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1582/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/223/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/198/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1036/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1039/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1065/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1218/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/4643/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/492/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1040/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1043/
DEBUG:root:The URL being read in

DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3778/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/228/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/645/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/1328/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3025/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3781/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3782/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/720/
DEBUG:root:The URL being read in from csv is: http://cmano-db.com/aircraft/3780/
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (m

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): cmano-db.com:80
DEBUG:urllib3.connectionpool:http://cmano-db.com:80 "GET /aircraft/1658/ HTTP/1.1" 200 None
DEBUG:root:Table 1 returned successfully.
DEBUG:root:Table 2 returned successfully.
DEBUG:root:Table 1 and 2 combined successfully.
DEBUG:root:Row 19 appended to dataframe.
DEBUG:root:Current URL being processed is: http://cmano-db.com/aircraft/4489/ at index: 20.
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): cmano-db.com:80
DEBUG:urllib3.connectionpool:http://cmano-db.com:80 "GET /aircraft/4489/ HTTP/1.1" 200 None
DEBUG:root:Table 1 returned successfully.
DEBUG:root:Table 2 returned successfully.
DEBUG:root:Table 1 and 2 combined successfully.
DEBUG:root:Row 20 appended to dataframe.
DEBUG:root:Current URL being processed is: http://cmano-db.com/aircraft/446/ at index: 21.
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): cmano-db.com:80
DEBUG:urllib3.connectionpool:http://cmano-db.com:8

IndexError: list index out of range